In [ ]:
def load_train_data():
    """
        for trainging data, get the feature for each word:
    """
    filename = 'SBD.train'
    SBD_traindata_list = []
    num =0
    with open(filename,'r') as f:
        for line in f:
            line = line.strip('\n')
            word = line.split(' ')
            SBD_traindata_list.append([word[0],word[1],word[2]])
            if word[2] =='EOS' or word[2]=='NEOS':
                num+=1
    print('num of all EOS NEOS')
    print(num)
    return SBD_traindata_list

In [1]:
import numpy as np
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder
from sklearn import tree


def load_train_data():
    """
        for trainging data, get the feature for each word:
    """
    filename = 'SBD.train'
    SBD_traindata_list = []
    num =0
    with open(filename,'r') as f:
        for line in f:
            line = line.strip('\n')
            word = line.split(' ')
            SBD_traindata_list.append([word[0],word[1],word[2]])
            if word[2] =='EOS' or word[2]=='NEOS':
                num+=1
    print('num of all EOS NEOS: '+str(num))

    return SBD_traindata_list

def load_test_data():
    """
        for test data, get the feature for each word:
    """
    filename = 'SBD.test'
    SBD_testdata_list = []
    num = 0
    with open(filename,'r') as f:
        for line in f:
            line = line.strip('\n')
            word = line.split(' ')
            SBD_testdata_list.append([word[0],word[1],word[2]])
            if word[2] =='EOS' or word[2]=='NEOS':
                num+=1
    print('num of all EOS NEOS: '+str(num))
    return SBD_testdata_list

SBD_traindata_list = load_train_data()
SBD_testdata_list = load_test_data()
# print(len(SBD_traindata_list))
# print(len(SBD_testdata_list))


def feature_label(data_list):
    feature = []
    label = []
    index = 0
    for pair in data_list:
        if pair[2] == 'EOS' or pair[2] == 'NEOS':
            if index > 0:
                L = data_list[index - 1][1]
                num = 1
                while str.isalpha(L) == False:
                    num += 1
                    L = data_list[index - num][1]

            if index != len(data_list) - 1:
                R = data_list[index + 1][1]
                num = 1
                while str.isalpha(R) == False:
                    num += 1
                    R = data_list[index + num][1]

            len_L = len(L) < 3
            L_Cap = L.isupper()
            R_Cap = R.isupper()
            feature.append([L, R, len_L, L_Cap, R_Cap])

            if pair[2] =='EOS':
                label.append(1)
            else:
                label.append(0)

        index += 1
    return feature, label

train_feature, train_label = feature_label(SBD_traindata_list)
test_feature, test_label = feature_label(SBD_testdata_list)
# print(len(train_feature))
# print(len(test_feature))

# encode feature vector of
def encode_feature(train_feature,test_feature):

    # label the left and right english vocab:
    word_dict = {}
    for vector in train_feature:
        if vector[0] not in word_dict:
            word_dict[vector[0]] = 1
        if vector[1] not in word_dict:
            word_dict[vector[1]] = 1
    for vector in test_feature:
        if vector[0] not in word_dict:
            word_dict[vector[0]] = 1
        if vector[1] not in word_dict:
            word_dict[vector[1]] = 1
    print("num of word: " + str(len(word_dict)))

    # one hot english vocab:
    word_list = []
    for word, val in word_dict.items():
        word_list.append(word)
    enc = preprocessing.LabelEncoder()
    enc.fit(word_list)
    word_index = enc.transform(word_list)
    # print(word_index) # [1 2 0]
    word_index = word_index.reshape(-1, 1)  # Needs to be the correct shape
    ohe = preprocessing.OneHotEncoder(sparse=False)  # Easier to read
    word_feature = ohe.fit_transform(word_index)
    num = 0
    for word in word_list:
        word_dict[word] = word_feature[:, num]
        num += 1

    len_onehot = len(word_feature)

    # lable each feature vector
    OH_train_feature = []
    for vector in train_feature:
        new_feature = []
        new_feature.extend(word_dict[vector[0]])
        new_feature.extend(word_dict[vector[1]])
        new_feature.append(int(vector[2]))
        new_feature.append(int(vector[3]))
        new_feature.append(int(vector[4]))
        OH_train_feature.append(new_feature)

    OH_test_feature = []
    for vector in test_feature:
        new_feature = []
        new_feature.extend(word_dict[vector[0]])
        new_feature.extend(word_dict[vector[1]])
        new_feature.append(int(vector[2]))
        new_feature.append(int(vector[3]))
        new_feature.append(int(vector[4]))
        OH_test_feature.append(new_feature)

    return OH_train_feature, OH_test_feature

OH_train_feature, OH_test_feature = encode_feature(train_feature,test_feature)

# print(len(OH_train_feature[0]))
# print(len(OH_test_feature))
# print(np.shape(OH_test_feature))
# print(np.shape(test_label))
#
# clf = tree.DecisionTreeClassifier()
# clf = clf.fit(OH_train_feature, train_label)

# print(len(OH_train_feature[0]))
# # Y = [0, 1]
# # clf = tree.DecisionTreeClassifier()
# # clf = clf.fit(X, Y)









num of all EOS NEOS: 17224
num of all EOS NEOS: 8370
num of word: 8142


In [3]:
clf = tree.DecisionTreeClassifier()
clf = clf.fit(OH_test_feature, test_label)

# predict
test_result = clf.predict(OH_test_feature)
num_sample = len(test_result)
num = 0
index = 0
for result in test_result:
    if result == test_label[index]:
        num+=1
    index+=1
print(num/num_sample)

0.953763440860215


In [376]:
def load_test_data():
    """
        for test data, get the feature for each word:
    """
    filename = 'SBD.test'
    SBD_testdata_list = []
    num = 0
    with open(filename,'r') as f:
        for line in f:
            line = line.strip('\n')
            word = line.split(' ')
            SBD_testdata_list.append([word[0],word[1],word[2]])
            if word[2] =='EOS' or word[2]=='NEOS':
                num+=1
    print('num of all EOS NEOS')
    print(num)
    return SBD_testdata_list

In [377]:
SBD_traindata_list = load_train_data()
SBD_testdata_list = load_test_data()
print(len(SBD_traindata_list))
print(len(SBD_testdata_list))

num of all EOS NEOS
17224
num of all EOS NEOS
8370
297255
148647


In [387]:
def feature_label(data_list):
    feature = []
    label=[]
    index = 0
    for pair in data_list:
        if pair[2] =='EOS' or pair[2] =='NEOS':
            if index >0:
                L = data_list[index-1][1]
                num = 1 
                while str.isalpha(L) == False:
                    num+=1
                    L = data_list[index-num][1]
                
            if index !=len(data_list)-1:
                R = data_list[index+1][1]
                num = 1 
                while str.isalpha(R) == False:
                    num+=1
                    R = data_list[index+num][1]

            len_L = len(L)<3
            L_Cap = L.isupper()
            R_Cap = R.isupper()
            feature.append([L,R,len_L,L_Cap,R_Cap])
            label.append(pair[2])
        index+=1
    return feature,label

In [389]:
train_feature, train_label = feature_label(SBD_traindata_list)
test_feature, test_label = feature_label(SBD_testdata_list)
print(len(train_feature))
print(len(test_feature))

17224
8370


In [390]:
train_feature

[['Kent', 'for', False, False, False],
 ['the', 'We', False, False, False],
 ['the', 'Kissinger', False, False, False],
 ['his', 'So', False, False, False],
 ['do', 'Kissinger', True, False, False],
 ['Jennings', 'Should', False, False, False],
 ['the', 'impose', False, False, False],
 ['sanctions', 'Kissinger', False, False, False],
 ['the', 'and', False, False, False],
 ['Chinese', 'A', False, False, True],
 ['later', 'Kissinger', False, False, False],
 ['economic', 'No', False, False, False],
 ['eight', 'Most', False, False, False],
 ['that', 'Kissinger', False, False, False],
 ['Chinese', 'The', False, False, False],
 ['itself', 'Nonetheless', False, False, False],
 ['elicited', 'ABC', False, False, True],
 ['of', 'Kissinger', True, False, False],
 ['to', 'If', True, False, False],
 ['simple', 'Jennings', False, False, False],
 ['yesterday', 'And', False, False, False],
 ['that', 'On', False, False, False],
 ['Wednesday', 'Kissinger', False, False, False],
 ['CBS', 'and', False, Tr

In [391]:
from sklearn import tree

clf = tree.DecisionTreeClassifier()
clf = clf.fit(train_feature, train_label)


ValueError: could not convert string to float: 'Kent'

In [398]:
from sklearn import preprocessing
from sklearn.preprocessing import OrdinalEncoder
enc = preprocessing.OrdinalEncoder()

X = [['male', 'from US', 'uses Safari'], ['female', 'from Europe', 'uses Firefox']]
enc.fit(X)  


ImportError: cannot import name 'OrdinalEncoder'

In [399]:
from sklearn.preprocessing import OrdinalEncoder

ImportError: cannot import name 'OrdinalEncoder'

In [400]:
from sklearn.preprocessing import CategoricalEncoder

ImportError: cannot import name 'CategoricalEncoder'

In [ ]:
enc = preprocessing.LabelEncoder()
enc.fit(word_list)